Author: Kyle Hsu

With revisions from: Rafael Rafailov, Evan Liu, Fahim Tajwar

# Introduction
Welcome to the PyTorch tutorial for XCS330! This colab notebook accompanies [these slides](https://docs.google.com/presentation/d/1e_md1C24vZsMNtbAJyNyDkau99owklm65IG9MudizvE/edit?usp=sharing). If you haven't already, enable a GPU for this colab instance by doing "Edit" -> "Notebook settings" -> "Hardware accelerator" drop-down -> "GPU" -> "Save".

Let's make sure we're using the right Python and PyTorch versions, and that we have a GPU at our disposal.

In [ ]:
import sys
import torch
print(f'Python version info: {sys.version}')
print(f'PyTorch version info: {torch.__version__}')
print(f'PyTorch detects a GPU: {torch.cuda.is_available()}')

Python version info: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
PyTorch version info: 2.1.0+cu121
PyTorch detects a GPU: True


# Overview
We'll begin by doing some MNIST classification, starting from low-level operations and gradually replacing chunks using `torch` abstractions such as `torch.nn.Module`, `torch.optim.SGD`, and `torch.utils.data.DataLoader`. (Much of this content is adapted from [this tutorial](https://pytorch.org/tutorials/beginner/nn_tutorial.html) by Jeremy Howard.) We'll then proceed to a whirlwind tour over PyTorch features that we suspect will be relevant for your homeworks for this course. Finally, we'll see some tips and tricks for debugging and getting help.

# PyTorch Basics via MNIST
### MNIST
Download MNIST.

In [ ]:
from pathlib import Path
import requests

DATA_PATH = Path("./data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "https://github.com/pytorch/tutorials/raw/main/_static/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

Load MNIST into memory.

In [ ]:
import pickle
import gzip
import numpy as np

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_val, y_val), _) = pickle.load(f, encoding="latin-1")

The data currently exists as NumPy arrays. Let's take a closer look.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

print(f'training inputs: shape {x_train.shape}, dtype {x_train.dtype}')
print(f'training outputs: shape {y_train.shape}, dtype {y_train.dtype}')
print(f'input range: {x_train.min(), x_train.max()}')
print(f'label range: {y_train.min(), y_train.max()}')

i_example = 42
x_train_example = x_train[i_example].reshape((28, 28))
y_train_example = y_train[i_example]
plt.imshow(x_train_example, cmap='gray')
print(f'label of training example {i_example}: {y_train_example}')

We need to convert the data into PyTorch tensors. For ease of use, tensors have some familiar attributes and methods.

In [ ]:
x_train, y_train, x_val, y_val = map(torch.tensor, (x_train, y_train, x_val, y_val))
print(f'training inputs: shape {x_train.shape}, dtype {x_train.dtype}')
print(f'training outputs: shape {y_train.shape}, dtype {y_train.dtype}')
print(f'input range: {x_train.min(), x_train.max()}')
print(f'label range: {y_train.min(), y_train.max()}')

### Bare Metal MNIST
We will first train a model for classifying MNIST digits with nothing but tensor and gradient operations. We'll later substitute out chunks of this code with the appropriate PyTorch abstractions.

To keep things simple, for this we'll use a linear model. For a single example, the model is
$$ z = W^\top x + b $$
where $W \in \mathbb{R}^{784 \times 10}$ and $b \in \mathbb{R}^{10}$ are the parameters to be learned, $x \in [0, 1]^{784}$ is the MNIST image, and $z \in \mathbb{R}^{10}$ are the logits of the categorical distribution for the label of $x$.

To obtain categorical parameters from the logits, we do
$$ p = \text{softmax}(z) = \frac{1}{\sum_j \exp(z_j)} \exp(z).$$

The label $y \in \{0, \dots, 9\}$ implicitly defines a one-hot categorical parameter vector $y' \in \{0, 1\}^{10}$, where $y$ corresponds to the index of $y'$ that is 1. The negative log-likelihood is then
$$ \text{nll}(p, y) = - \sum_{i=0}^{9} y'_i \log(p_i) = -\log(p_y)$$

Note that our implementation will assume batched inputs with a batch size of $B$. That is,
$$x_b = \begin{bmatrix} x_1^\top \\ \vdots \\ x_B^\top \end{bmatrix} \in [0, 1]^{B \times 784}.$$ We can vectorize the above operations to process all $B$ inputs at once (naively using `for` loops is much, much slower). For example, to compute the logits, we have
$$z_b = x_b W + \begin{bmatrix} b^\top \\ \vdots \\ b^\top \end{bmatrix} \in \mathbb{R}^{B \times 10}$$

In [ ]:
W = torch.randn(784, 10, requires_grad=True)
b = torch.randn(10, requires_grad=True)

def logits(x_b, W, b):
    return x_b @ W + b    # addition is broadcasted

def log_softmax(z_b):
    return z_b - torch.logsumexp(z_b, dim=1, keepdim=True)  # combining log and softmax is more numerically stable

def negative_log_likelihood(logp_b, y_b):
    return -torch.mean(logp_b[range(y_b.shape[0]), y_b])   # indexing trick

def accuracy(logit_b, y_b):
    return (logit_b.argmax(dim=1) == y_b).float().mean().item()

Note that the `requires_grad` keyword argument/attribute determines whether a tensor is a constant or a variable.

In [ ]:
print(b.requires_grad)
print(y_train.requires_grad)

### Computation Graph
Now we have the opportunity to look at some of PyTorch's core mechanics: reverse-mode automatic differentiation (a.k.a. backpropagation) on a dynamic computation graph. Let's take a batch of training data and compute the average loss over the batch.

In [ ]:
batch_size = 64
x_b, y_b = x_train[:batch_size], y_train[:batch_size]

def loss_fn(W, b, x_b, y_b):
    logit_b = logits(x_b, W, b)
    logp_b = log_softmax(logit_b)
    loss = negative_log_likelihood(logp_b, y_b)
    return loss

loss = loss_fn(W, b, x_b, y_b)

Computations like the above are automagically recorded on a computation graph. Let's take a peek.

In [ ]:
!pip install torchviz

In [ ]:
import torchviz
torchviz.make_dot(loss)

We can actually inspect tensors to see that each non-leaf variable tensor records its gradient function based on how it was obtained.

In [ ]:
print(b.grad_fn)    # leaf variable
print(loss.grad_fn)
print(x_b.grad_fn)  # not a variable

### Using `autograd.grad` to Compute Gradients
We can now use reverse-mode automatic differentiation to compute the gradient of the loss with respect to our parameters!

In [ ]:
from torch import autograd
loss = loss_fn(W, b, x_b, y_b)
W_grad, b_grad = autograd.grad(loss, inputs=(W, b))
print(b_grad)

We're now ready to optimize our linear MNIST model. Each iteration consists of the following steps:
* Sample a batch of training data.
* Compute the logits from the inputs using the model parameters.
* Compute a scalar loss from the logits and labels.
* Compute the gradient of the loss with respect to the parameters.
* Update the model parameters using the gradient.

Note that the last step is done in a context manager that disables the construction of the computation graph.

To check whether our optimization was successful, we'll look at the accuracy on the training data before and after.

In [ ]:
lr = 0.5
num_epochs = 2

def train(W, b):
    print(f'accuracy before: {accuracy(logits(x_train, W, b), y_train)}')

    for i_epoch in range(num_epochs):
        i_batch_start = 0
        while i_batch_start + batch_size < x_train.shape[0]:
            x_b = x_train[i_batch_start:i_batch_start + batch_size]
            y_b = y_train[i_batch_start:i_batch_start + batch_size]
            i_batch_start += batch_size

            logit_b = logits(x_b, W, b)
            logp_b = log_softmax(logit_b)
            loss = negative_log_likelihood(logp_b, y_b)
            W_grad, b_grad = autograd.grad(loss, inputs=(W, b))

            with torch.no_grad():   # we don't need gradients for this
                W -= lr * W_grad
                b -= lr * b_grad

    print(f'accuracy after: {accuracy(logits(x_train, W, b), y_train)}')

W = torch.randn(784, 10, requires_grad=True)
b = torch.randn(10, requires_grad=True)
train(W, b)

### Using `backward` to Compute Gradients
An alternative to using `autograd.grad` is to call `backward()` on the loss tensor. This _accumulates_ gradients of leaf tensors into their `grad` attribute. (Note that with this we don't need to specify the variables we're taking the gradient with respect to, nor do we obtain the gradients as output from the function call.)

When we're done using a gradient in a `grad` attribute, we should reset it to 0, else repeated accumulations may occur. We can do this with the `Tensor` method `zero_()`. (Note that the `_` signifies that the operation modifies the tensor in-place.)

In [ ]:
print('before computing gradients')
print(b.grad)

loss = loss_fn(W, b, x_b, y_b)
loss.backward()
print('after forward pass and calling backward()')
print(b.grad)

loss = loss_fn(W, b, x_b, y_b)
loss.backward()
print('after another forward pass and calling backward() again')
print(b.grad)

b.grad.zero_()
print('after calling zero_()')
print(b.grad)

When should you use `autograd.grad` vs. `backward`? In general, `backward` is more convenient, but there are times (e.g. in Homework 2) when you need more fine-grained control over gradient computation. Let's see what the training code looks like when we use `backward`:

In [ ]:
def train(W, b):
    print(f'accuracy before: {accuracy(logits(x_train, W, b), y_train)}')

    for i_epoch in range(num_epochs):
        i_batch_start = 0
        while i_batch_start + batch_size < x_train.shape[0]:
            x_b = x_train[i_batch_start:i_batch_start + batch_size]
            y_b = y_train[i_batch_start:i_batch_start + batch_size]
            i_batch_start += batch_size

            logit_b = logits(x_b, W, b)
            logp_b = log_softmax(logit_b)
            loss = negative_log_likelihood(logp_b, y_b)
            loss.backward()

            with torch.no_grad():
                W -= lr * W.grad
                b -= lr * b.grad
                W.grad.zero_()
                b.grad.zero_()

    print(f'accuracy after: {accuracy(logits(x_train, W, b), y_train)}')

W = torch.randn(784, 10, requires_grad=True)
b = torch.randn(10, requires_grad=True)
train(W, b)

### `nn.functional`
Now, let's refactor and extend our training code by allowing ourselves to use other `torch` modules. We'll first look at `torch.nn`.

`nn.functional` (commonly abbreviated as just `F`) includes many useful functions, e.g. loss functions and activation functions. In particular, `F.cross_entropy` combines the `log_softmax` and `negative_log_likelihood` functions into one.

If you're used to using TensorFlow's [CategoricalCrossentropy loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy), note that `F.cross_entropy` takes integer labels, not one-hot labels

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

with torch.no_grad():
    logit_b = logits(x_b, W, b)
    print(negative_log_likelihood(log_softmax(logit_b), y_b))
    print(F.cross_entropy(logit_b, y_b))

### `nn.Module`
`nn.Module` is a useful class that corresponds loosely to an intuitive notion of a parameterized model. It
* registers `nn.Parameter`s, which are essentially variable tensors, as attributes
* holds as the `forward` method the model's forward pass

In [ ]:
class MNISTLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.W = nn.Parameter(torch.randn(784, 10))
        self.b = nn.Parameter(torch.randn(10))

    def forward(self, x_b):
        return x_b @ self.W + self.b

Using the `nn.Module` can be very convenient. For example, previously we had to manually zero the `grad` attribute of `W` and `b`. We can now use the `parameters` method, which returns an iterator over module parameters, and the `zero_grad` method, which zeros the `grad` attribute of every module parameter.

After substituting code for the model forward pass, the loss computation, the optimization step, and the gradient zeroing, the training code now looks like this:

In [ ]:
def train(model):
    print(f'accuracy before: {accuracy(model(x_train), y_train)}')

    for i_epoch in range(num_epochs):
        i_batch_start = 0
        while i_batch_start + batch_size < x_train.shape[0]:
            x_b = x_train[i_batch_start:i_batch_start + batch_size]
            y_b = y_train[i_batch_start:i_batch_start + batch_size]
            i_batch_start += batch_size

            logit_b = model(x_b)    # an nn.Module is callable
            loss = F.cross_entropy(logit_b, y_b)
            loss.backward()

            with torch.no_grad():
                for p in model.parameters():
                    p -= lr * p.grad
                model.zero_grad()

    print(f'accuracy after: {accuracy(model(x_train), y_train)}')

train(MNISTLinear())

`nn` contains many `nn.Module` subclasses that correspond to commonly used chunks of computation. One such "layer" is the linear model we've been using! We can replace our `MNISTLinear` class with an `nn.Linear`.

In [ ]:
train(nn.Linear(784, 10))

Our optimization went differently in 2 epochs this time around. This is probably because the parameters in `nn.Linear` are [initialized slightly differently](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html). It's always important to read documentation!

### MNIST MLP
What if we want to use a two-layer fully-connected network? `nn.Module`s can register `nn.Module` attributes!

In [ ]:
class MNISTMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(784, 64)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(64, 10)

    def forward(self, x_b):
        return self.linear2(self.activation(self.linear1(x_b)))

train(MNISTMLP())

##Saving/Loading pre-trained weights
Sometimes multiple people may have to use the same network. It can take a lot of time for everyone to train their network from scratch. It is environmentally also disastrous as the carbon footprint of training large networks is very huge.

It would be better if we could train and save weights for our network, and reuse them later if we need to. PyTorch gives us ways to do this easily!

In [ ]:
network = MNISTMLP()
train(network)

torch.save(network.state_dict(), "network.pt")

The above code saves the network's weights to the given path, in this case "network.pt". Now we will show that loading the model's weight without retraining it from scratch retrieves prior performance.

In [ ]:
network = MNISTMLP()
print(f'accuracy of initialized network: {accuracy(network(x_train), y_train)}')

network.load_state_dict(torch.load("network.pt"))
print(f'accuracy after loading weights: {accuracy(network(x_train), y_train)}')

### Are You Registered?
`nn.Module`s [have some specific rules](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) regarding what attributes are properly registered (i.e. identified as part of the model). We've seen that `nn.Parameter`s and `nn.Module`s are, but most Python objects are not. For example, you might consider it cleaner to implement the `MNISTMLP` this way:

In [ ]:
class MNISTMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = [
            nn.Linear(784, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        ]

    def forward(self, x_b):
        for layer in self.layers:
            x_b = layer(x_b)
        return x_b

train(MNISTMLP())
print(list(MNISTMLP().parameters()))

Uh oh! Even though the `layers` list has `nn.Module`s, it itself wasn't recognized as a registerable attribute. To get around this, we have the `nn.ModuleList`:

In [ ]:
class MNISTMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Linear(784, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        ])

    def forward(self, x_b):
        for layer in self.layers:
            x_b = layer(x_b)
        return x_b

train(MNISTMLP())

### `nn.Sequential`
It's pretty common to have a sequence of layers we want to iteratively apply onto an input. We can replace the `for` loop business in the forward pass by using an `nn.Sequential` container, a very commonly used abstraction:

In [ ]:
train(nn.Sequential(
    nn.Linear(784, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
))

### `torch.optim`
`torch.optim` contains implementations of many common optimization algorithms. When initializing an optimizer instance, we need to pass in the model parameters. Now, instead of manually coding the update rule, we call the optimizer's `step` method. The optimizer can also handle resetting parameter gradients.

In [ ]:
from torch import optim

def train(model):
    optimizer = optim.SGD(model.parameters(), lr=lr)

    print(f'accuracy before: {accuracy(model(x_train), y_train)}')

    for i_epoch in range(num_epochs):
        i_batch_start = 0
        while i_batch_start + batch_size < x_train.shape[0]:
            x_b = x_train[i_batch_start:i_batch_start + batch_size]
            y_b = y_train[i_batch_start:i_batch_start + batch_size]
            i_batch_start += batch_size

            logit_b = model(x_b)
            loss = F.cross_entropy(logit_b, y_b)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

    print(f'accuracy after: {accuracy(model(x_train), y_train)}')

train(MNISTMLP())

### `torch.utils.data`
Notice how we are manually slicing the `x_train` and `y_train` tensors to get batches. There's a more convenient way to do this:
`torch.utils.data` features the `Dataset`, `Sampler`, and `DataLoader` abstractions, which are useful for transforming, sampling, and iterating over data. Let's implement a custom `Dataset` subclass for MNIST which we can then use to instantiate a `DataLoader`:

In [ ]:
from torch.utils import data

class MNISTDataset(data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

dataset_train = MNISTDataset(x_train, y_train)
dataloader_train = data.DataLoader(dataset_train, batch_size=batch_size)

def train(model):
    optimizer = optim.SGD(model.parameters(), lr=lr)

    print(f'accuracy before: {accuracy(model(x_train), y_train)}')

    for i_epoch in range(num_epochs):
        for x_b, y_b in dataloader_train:   # dataloaders are iterators
            logit_b = model(x_b)
            loss = F.cross_entropy(logit_b, y_b)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

    print(f'accuracy after: {accuracy(model(x_train), y_train)}')

train(MNISTMLP())

### Using a GPU
Now, let's see how to use a GPU in PyTorch. Every tensor has a `device` attribute (default: `'cpu'`). GPU-accelerated computation requires the involved tensors to all be on device `'cuda'`. The `to` method is a handy way to move tensors around.



In [ ]:
x = torch.tensor(1.)
print(x.device)
x = x.to('cuda')
print(x.device)

The `nn.Module` also has a `to` method that transfers all parameters (and buffer items) to a specified device.

In [ ]:
from torch.utils import data

dataset_train = data.TensorDataset(x_train, y_train)
dataloader_train = data.DataLoader(dataset_train, batch_size=batch_size)

def train(model):
    optimizer = optim.SGD(model.parameters(), lr=lr)

    print(f'accuracy before: {accuracy(model(x_train.to("cuda")), y_train.to("cuda"))}')

    for i_epoch in range(num_epochs):
        for x_b, y_b in dataloader_train:
            x_b, y_b = x_b.to('cuda'), y_b.to('cuda')
            logit_b = model(x_b)
            loss = F.cross_entropy(logit_b, y_b)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

    print(f'accuracy after: {accuracy(model(x_train.to("cuda")), y_train.to("cuda"))}')

train(MNISTMLP().to('cuda'))

### Clean-up
To wrap this section up, we'll refactor our code to facilitate assessing on both train and val data with data-loading, and using the GPU.

In [ ]:
dataset_val = data.TensorDataset(x_val, y_val)
dataloader_val = data.DataLoader(dataset_val, batch_size=batch_size)

def loss_fn(model, x_b, y_b):
    logit_b = model(x_b)
    loss = F.cross_entropy(logit_b, y_b)
    return loss, y_b.shape[0]

def accuracy(model, x_b, y_b):
    logit_b = model(x_b)
    accuracy = (logit_b.argmax(dim=1) == y_b).float().mean()
    return accuracy, y_b.shape[0]

def assess(model, dataloader):
    loss_total_r, correct_r, n_r = 0, 0, 0
    with torch.no_grad():
        for x_b, y_b in dataloader:
            x_b, y_b = x_b.to('cuda'), y_b.to('cuda')
            loss, n = loss_fn(model, x_b, y_b)
            acc, n = accuracy(model, x_b, y_b)
            loss_total_r += loss * n
            correct_r += acc * n
            n_r += n
    loss_total_r, correct_r = loss_total_r.item(), correct_r.item()
    return loss_total_r / n_r, correct_r / n_r

def train(model, optimizer):
    print('before')
    print(f'train loss and acc: {assess(model, dataloader_train)}')
    print(f'val loss and acc: {assess(model, dataloader_val)}')

    for i_epoch in range(num_epochs):
        for x_b, y_b in dataloader_train:
            x_b, y_b = x_b.to('cuda'), y_b.to('cuda')
            loss, _ = loss_fn(model, x_b, y_b)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

    print('after')
    print(f'train loss and acc: {assess(model, dataloader_train)}')
    print(f'val loss and acc: {assess(model, dataloader_val)}')

model = MNISTMLP().to('cuda')
optimizer = optim.SGD(model.parameters(), lr=lr)
train(model, optimizer)

# More PyTorch Features: A Whirlwind Tour

### MNIST CNN
2D convolutional layers expect an input of shape $(B, C, H, W)$ or `(batch, channels, height, width)`. For a batch of MNIST images, this is $(B, 1, 28, 28)$.

In [ ]:
class MNISTCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=20, kernel_size=(5, 5), stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=20, out_channels=50, kernel_size=(5, 5), stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Flatten(),
            nn.Linear(800, 500),
            nn.ReLU(),
            nn.Linear(500, 10)
        )

    def forward(self, x_b):
        return self.layers(x_b.view((-1, 1, 28, 28)))

model = MNISTCNN().to('cuda')
optimizer = optim.Adam(model.parameters(), lr=1e-3)
train(model, optimizer)

### MNIST LSTM
When constructed using the `batch_first=True` argument, `nn.LSTM`s expect inputs of shape $(B, L, D)$ or `(batch, sequence length, element size)`. We'll interpret the rows of the MNIST image as sequence elements, and compute the logits from the last LSTM output (i.e. wait for the LSTM to see the entire image).

In [ ]:
class MNISTLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.LSTM = nn.LSTM(input_size=28, hidden_size=64, num_layers=1, batch_first=True)
        self.linear = nn.Linear(64, 10)

    def forward(self, x_b):
        out, _ = self.LSTM(x_b.view((-1, 28, 28)))  # out contains outputs at each iteration over the sequence
        return self.linear(out[:, -1, :])   # only the out at the last iteration has seen the entire image

model = MNISTLSTM().to('cuda')
optimizer = optim.Adam(model.parameters(), lr=1e-3)
train(model, optimizer)

### `nn.Embedding`
Oftentimes we will have categorical data with too many categories to comfortably convert into one-hot encodings. Examples of this include tokens in NLP and $(x,y)$ positions on a grid. One common way to process this data is to use an `nn.Embedding`, or a table of dense vectors that can be indexed by the categorical data. The dense vectors can be optimized. `nn.Embedding` supports advanced indexing.

In [ ]:
embedding = nn.Embedding(num_embeddings=1000, embedding_dim=64)
print(embedding(torch.tensor(42)).shape)
print(embedding(torch.tensor(42)).requires_grad)
indices = torch.tensor([
    [0, 1, 42, 999],
    [1, 1, 2, 3]
])
print(indices.shape)
print(embedding(indices).shape)

If you're interested in NLP, a nice use of `nn.Embedding` and recurrent layers (`nn.GRU`) for natural language translation can be found in [this tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html).

### `torch.distributions`
Aside from conveniently providing commonly used methods (e.g. density/mass evaluation), one of the main features of this module is the use of the reparameterization trick to facilitate backpropagation through random samples to the underlying distribution parameters (and beyond).

In [ ]:
from torch import distributions
loc = torch.tensor(0., requires_grad=True)
scale = torch.tensor(1., requires_grad=True)
p = distributions.Normal(loc, scale)
x = p.rsample(torch.Size([5]))
print(x)    # grad_fn comes from reparameterization trick
y = -torch.mean(x**2)
y.backward()
print(scale.grad)

If we don't use `rsample`, we can't take gradients through the sampling step.

In [ ]:
x = p.sample(torch.Size([5]))
print(x)
y = -torch.sum(x**2)
y.backward()

# Debugging, Documentation, and Getting Help
Your assignments won't be in Jupyter notebooks, but in Python scripts. This means you won't have the interactivity of cell-based execution, but to the rescue comes `pdb`, the Python Debugger. Use it to

*   set breakpoints in the code to interactively inspect program elements: `import pdb; pdb.set_trace()`
*   automatically start a debugging session when an exception is thrown: `python -m pdb -c continue main.py`

Both will be super, super useful.

When something about PyTorch is confusing you, the first place to look is the [PyTorch documentation](https://https://pytorch.org/docs/stable/index.html). (The author of this tutorial consulted the documentation no fewer than 20 times when making it.) This is often the last place you need to look, but sometimes the [PyTorch forums](https://https://discuss.pytorch.org/) can be helpful, too. (Use the search tool!) And of course, you're always welcome to post on Ed or seek out one of your friendly neighborhood CAs.





---

